In [18]:
import os
from glob import glob
from tqdm import tqdm 

import numpy as np
import matplotlib.pyplot as plt 

import tifffile 
import json

In [19]:
root_dir = 'D:\sagar\Data'
scans = os.listdir(root_dir)

In [20]:
path_roi = []
path_notRoi = []

for s in scans:
    scan_path = os.path.join(root_dir, s)
    for r in os.listdir(os.path.join(scan_path, 'roi')):
        path_roi.append((os.path.join(root_dir, s, 'roi', r)))
    try:
        for nr in os.listdir(os.path.join(scan_path, 'not_roi')):
            path_notRoi.append((os.path.join(root_dir, s, 'not_roi', nr)))
    except:
        pass
        
    
    
    

In [23]:
len(path_roi)

533

In [24]:
len(path_notRoi)

561